In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 12134085
paper_name = 'bonangelino_bonifacino_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/hit_list.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 737 x 7


In [7]:
original_data.head()

,late Well Int. ORF Gene Aliases Description KnownVPS Genes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,380,E3,S,YKR001C,VPS1,"GRD1, LAM1, SPO15, VPL1, VPT26",Dynamin-like GTPase
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,346,C5,S,YDR495C,VPS3,"PEP6, VPL3, VPT17","Low abundance, hydrophilic protein"
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
original_data['orf'] = original_data['Unnamed: 3'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [15]:
typos = {'YKL054': 'YKL054C','YDR525W-': 'YDR525W-A'}
original_data['orf'] = original_data['orf'].apply(lambda x: typos[x] if x in typos.keys() else x)

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            late Well Int. ORF Gene Aliases Description KnownVPS Genes   \
index_input                                                               
0                                                          NaN            
2                                                          NaN            
4                                                          NaN            
6                                                          NaN            
8                                                          NaN            
...                                                        ...            
728                                                        NaN            
730                                                        NaN            
732                                                        NaN            
734                                                        NaN            
736                                                        NaN            

            Unnamed: 1 U

In [19]:
original_data = original_data.loc[t,:]

In [21]:
data_switch = {'W': -1, 'M': -2, 'S': -3, 'W-M': -1.5}
original_data['data'] = original_data['Unnamed: 2'].apply(lambda x: data_switch[x.strip()])

In [23]:
original_data.set_index('orf', inplace=True)

In [24]:
original_data = original_data[['data']].copy()

In [25]:
original_data = original_data.groupby(original_data.index).mean()

In [26]:
original_data.shape

(361, 1)

# Prepare the final dataset

In [27]:
data = original_data.copy()

In [28]:
dataset_ids = [469]
datasets = datasets.reindex(index=dataset_ids)

In [29]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [30]:
data.head()

dataset_id,469
data_type,value
orf,
YAL002W,-3.0
YAL004W,-1.0
YAL009W,-1.0
YAL011W,-1.0
YAL014C,-1.0


## Subset to the genes currently in SGD

In [31]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [32]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,469
,data_type,value
gene_id,orf,
2,YAL002W,-3.0
1863,YAL004W,-1.0
7,YAL009W,-1.0
9,YAL011W,-1.0
12,YAL014C,-1.0


# Normalize

In [33]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [34]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [35]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        469          
data_type       value    valuez
gene_id orf                    
2       YAL002W  -3.0 -6.345088
1863    YAL004W  -1.0 -2.115029
7       YAL009W  -1.0 -2.115029
9       YAL011W  -1.0 -2.115029
12      YAL014C  -1.0 -2.115029

# Print out

In [36]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [37]:
from IO.save_data_to_db3 import *

In [38]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 12134085...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.24it/s]

Updating the data_modified_on field...
